In [1]:
import ee
import ee.mapclient
import time
import datetime
import sys
!pip install geetools

     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 225kB 7.5MB/s 
  Created wheel for geetools: filename=geetools-0.6.0-cp36-none-any.whl size=81649 sha256=d1d14457a1b98546def17576d71f4c310325c4e15c4c995b81128dc9eb249ab3
  Stored in directory: /root/.cache/pip/wheels/8b/0d/90/c214fd045df9bfc0e9f15dd5819ad4038fc90fab431e77da72
  Created wheel for pyshp: filename=pyshp-2.1.0-cp36-none-any.whl size=32609 sha256=aa037ccb4feedac989bc73662da19ec7de0068fa99241b95066d23a0f7608ed4
  Stored in directory: /root/.cache/pip/wheels/a6/0c/de/321b5192ad416b328975a2f0385f72c64db4656501eba7cc1a
Successfully built geetools pyshp


In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=GqZkJyz7Lq6Mz-gIZHjvp7QyTJCy4SDo7zsxR8RI8Kc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3QGXDRRsrHlqBJwNpjfcQaHU2LXcl7n9l2H4QQQ-VRWnvb0s8lb3rFc

Successfully saved authorization token.


In [3]:
# Method for processing Sentinel-2 dataset
def process_sentinel(dataset, time_range, area):

    # remove clouds from images with masking
    def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_m = ee.Number(2).pow(10).int()
        cirrus_m = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_m).eq(0) and(
            band_qa.bitwiseAnd(cirrus_m).eq(0))
        return image.updateMask(mask).divide(10000)

    # produce filtered image using median
    filter_image = (ee.ImageCollection(dataset).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).
                         map(maskclouds))

    sentinel_median = filter_image.median()
    # image_band = sentinel_median.select(['B4','B3','B2'])
    return sentinel_median

In [6]:
# Method for processing FLDAS/GLDAS dataset 
def process_FLDAS(dataset, timerange, coordinates, band):
    collection = (ee.ImageCollection(dataset)
            .filterDate(timerange[0], timerange[1])
            .filterBounds(coordinates)
            );
    
    image = collection.first()

    return image.clip(coordinates)

### Download Data
Accepts a list of datasets and output polygon regions and exports images to Drive folder. Only accepts ImageCollection datasets.

In [16]:
def download_data(datasets, timerange, coordinates, outputFolder, scale, report=False, testRun=True):
      
  print("Exporing Image to " + outputFolder + " Folder")

  for d_i,d in enumerate(datasets):
    
    print("\nProcessing " + d + "...")
    
    for c_i, c in enumerate(coordinates):

      if d == "COPERNICUS/S2" or d == "COPERNICUS/S2_SR":
        image = process_sentinel(d, timerange, c)
      elif d == "NASA/FLDAS/NOAH01/C/GL/M/V001" or d == "NASA/GLDAS/V021/NOAH/G025/T3H":
        # SELECT BAND
        image = process_FLDAS(d, timerange, c, 'AvgSurfT_inst')
      else:
        collection = (ee.ImageCollection(d)
                .filterDate(timerange[0], timerange[1])
                .filterBounds(c)
                )
        image = collection.first().clip(c)

      if testRun:
        print("Dataset: " + str(d_i) + ", Coordinates: " + str(c_i))
      else:
        print("Sending img_"+ str(d_i) + "_" + str(c_i)+" to GEE...", end=" ")
        task = ee.batch.Export.image.toDrive(
            image=image,
            description="img_" + str(d_i) + "_" + str(c_i),
            folder="testing_ee",
            region=c,
            scale=30
        )
        task.start()
        print("\x1b[32mComplete\x1b[0m")

    print("Complete")

### Image download 
Use function if downloading image rather than image collection

In [18]:
def download_data_image(datasets, timerange, coordinates, outputFolder, scale, report=False, testRun=True):

  image = (ee.Image(dataset))
  image = image.clip(coordinates)

  task = ee.batch.Export.image.toDrive(
          image=image,
          description="image",
          folder="testing_ee",
          scale=30,
          
      )
  task.start()
  print("Complete")
    

### Generate Polygon
Takes a center coordinate and box size and returns a polygon for cropping

In [11]:
# Method to generate polygons from center points and grid sizes
def generate_polygons(coordinates, boxSize):
  polygons = []
  for c in coordinates:
    bw = boxSize[0]/2
    bh = boxSize[1]/2
    wc = c[0]
    hc = c[1]

    polygon = ee.Geometry.Polygon([
      [[wc-bw, hc-bh],
      [wc+bw, hc-bh],
      [wc+bw, hc+bh],
      [wc-bw, hc+bh],
      [wc-bw, hc-bh]
      ]
    ])

    polygons.append(polygon)
  print(polygons)
  return polygons

### Call Functions
Example of calling download_data function

In [ ]:
datasets = ['CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics', 
            'NASA/FLDAS/NOAH01/C/GL/M/V001', 
            'NASA/GLDAS/V021/NOAH/G025/T3H', 
            'JAXA/ALOS/PALSAR/YEARLY/FNF',
            'COPERNICUS/Landcover/100m/Proba-V/Global',
            'MODIS/006/MCD12Q1',
            'NASA/MEASURES/GFCC/TC/v3',
            'CIESIN/GPWv411/GPW_Population_Density'
            ]

# Date range, datetime(year, month, day)
dates = [datetime.datetime(2010, 1, 1), datetime.datetime(2019, 5, 1)]

# Center coordinate and box size
centers = [[47.5207953674, -18.9101632941]]
box = [0.2, 0.2]


download_data(datasets=datasets, timerange=dates, coordinates=generate_polygons(centers, box) , outputFolder='testing_ee', scale=1, testRun=False)